The purpose of the notebook is the following:

0) Receive a dataset containing an aggregate dataframe with acne severities for a group of patients
1) Parse the data and seperate it into one dataframes per patient. Raw acne severities are converted into % change in acne severity relative to average baseline. The average baseline is computed dynamically for each patient.
2) Add a treatment history metadata column to each patient dataframe in the form ( (Treatment a1, Day 1), ..., (Treatment an, Day i)) where n is the index of a given treatment in the full history.
3) Produce the distribution of % changes in acne severity over all histories and patients. Fit a kernel density estimate and display it for visual inspection.
4) (Assumes a bimodal distribution); Optimize the KDE for local maxima and saddle point; find quantiles corresponding to local maxima and saddle points. Each quantile defines an acne severity change state. **In progress: dynamic decision of state number.*
5) Assign an acne severity change state to each acne severity entry for all patient dataframes.       
6) (Assumes a Dirichlet prior distribution for acne severity change probabilities) Calculate posterior distributions of acne severity change states for each consecutive treatment history. Plot each posterior as a striped heatmap for visual inspection. **In progress: dynamic decision of prior.*
8) Calculate Kullback-Leibler Divergence between consecutive treatment history posteriors. Plot stepwise and cumulative KL Divergence for visual inspection. cumulative KL Divergence is interpreted as cumulative information gain from changing acne severity state distributions. 
9) Approximate concavity of cumulative KL Divergence vs. treatment history ordinal curve; use user provided percentile cutoff to define inflection point concavity threshold.
10) Partion cumulative KL Divergence array between inflection points. Prevents regression model overfitting by checking if slope difference between adjacent inflection points exceeds user provided threshold. **In progress: further pruning of inflection points with AIC/BIC.*
11) Fits linear regression model to each cumulative KL Divergence sublist; returns slope of each. Quantifies extent of change in acne severity state distribution pairwise. **In progress: Plotting treatment history vs. inflection point slope to reveal treatments responsible for greatest changes.*


In [522]:
#virtual environment 
! pip install matplotlib pandas scipy numpy seaborn scikit-learn

In [523]:
#imports

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle as rect
import pandas as pd
import scipy as sp
from scipy import optimize
import numpy as np
import seaborn as sns
from collections import defaultdict, Counter
from matplotlib.cm import viridis
from scipy.stats import dirichlet 
from scipy.stats import beta
from scipy.special import gammaln, psi
from scipy.ndimage import gaussian_filter1d
from sklearn.linear_model import LinearRegression

In [524]:
def seperate_patients(raw_data):
    """Function that separates the raw dataframe via the following:
    1) Constructs an array tracking where the original date (2018-01-01) recurs.
    2) Uses that array to split raw_data into seperate dataframes.
    """
    #splitting data into different patients using leftmost column index (this is an assumption however)
    seperatePatientsIndices = list(raw_data.index[raw_data["date"] == "2018-01-01"])
    seperatePatientsIndices.append(len(raw_data))

    #checking to see if, after the same number of days in all dataframes, treatment of some sort was introduced
    #verifies that on day 29 
    #then adds the days each additional treatment was added after the fact (turns out they are all the same too)
    allPatientsIntroDays = []
    
    #seperating single dataframe into list of dataframes for each patient
    startIndex = 0
    seperatePatientsDFs = []
    for endIndex in seperatePatientsIndices[1:]:
        seperatePatientsDFs.append(raw_data[startIndex:endIndex])
        startIndex = endIndex
     
    for seperatePatientDF in seperatePatientsDFs:
        treatmentIntroDays =  []
        currentTreatment = seperatePatientDF["treatment"].iloc[0]
          
        for lineIndex in range(1,len(seperatePatientDF)): 
            if seperatePatientDF["treatment"].iloc[lineIndex] != currentTreatment:
                treatmentIntroDays.append([currentTreatment, "end day is", lineIndex])
                currentTreatment  = seperatePatientDF["treatment"].iloc[lineIndex]
                
                
        #remembering the end day of the last treatment and appending to the list
        lastTreatment  = seperatePatientDF["treatment"].iloc[len(seperatePatientDF)-1]
        treatmentIntroDays.append( [lastTreatment, "end day is", len(seperatePatientDF) - 1 ] )
         
        allPatientsIntroDays.append(treatmentIntroDays)
        
    return seperatePatientsDFs, allPatientsIntroDays

In [525]:
def add_history_metadata(seperatePatientsDFs, allPatientsIntroDays):

    """Function that adds a treatment history metadata column to each patient's dataframe by...
    1) Loading each seperate patient's dataframe and compute the average baseline severity, normalizing acne severity scores. Then modifies the dataframe, called a severities dataframe.
    2) For each, mapping each value for treatment to the a treatment history tuple. It is a tuple of the form ((days of treatment, ai), (days of treatment, ai+1),....(days of treatment, an))
    where n is the row number of the current day of the particular treatment."""
    
    
    #initializing dict containing severties by day for a given treatment 
    severitiesDayTreatmentDict = {treatment: None for treatment in seperatePatientsDFs[0]["treatment"]}
    
    modifiedDFs = []
    counter = 0 

    for patient_DF, days_of_intro in zip(seperatePatientsDFs, allPatientsIntroDays):
        #computing average baseline severity for each DF
        average_bl = patient_DF["AcneSeverity"].head(days_of_intro[0][2]).mean()
        
        #forming new dataframe from old one containing percent severity over baseline 
        modified_DF = patient_DF.copy()
        
        
        modified_DF["AcneSeverity"] = modified_DF["AcneSeverity"].apply(lambda x: (x - average_bl)/average_bl)*100
        modifiedDFs.append(modified_DF)
        counter += 1
    
    metadata_DFs = []
    #iterating over all dataframes and their respective treatment days of introduction
    for severities_df, days_of_intro in zip (modifiedDFs, allPatientsIntroDays): 
        #for each dataframe and the corresponding set of days where a given treatment ends
        
        #adding an explicit day column to each severities dataframe to enable indexing with df.loc  
        severities_df["day"] = range(len(severities_df))
    
        #initializing the treatment history metadata column
        severities_df["treatment_history"] = None
    
        last_treat_index = 0 #keeps track of which number of the ordered list of treatments is currently being processed
        treatment_days = {} #keeping track of how many days each particular treatment goes on for
        treatment_history = [] #keeping track of the full history of which treatment occurs before the others and how long they last for
    
        #also keeping track of the last treatment
        last_treatment_itself = None
        
        #iterating through rows of the dataframe
        for row_index, row in severities_df.iterrows():
            current_day = row["day"]
            current_treatment = row["treatment"]
                   
            #also checking to see if the current treatment is entirely new or falls inside a different treatment block
            if current_treatment != last_treatment_itself:
                treatment_days[current_treatment] = 1  # First day of new treatment
                treatment_history.append((current_treatment, 1))
            else:
                treatment_days[current_treatment] += 1
                treatment_history[-1] = (current_treatment, treatment_days[current_treatment])
                   
           
            #once history is built, the history is stored in the original dataframe as an entry in own column
            severities_df.at[row_index, "treatment_history"] = list(treatment_history)
    
            last_treatment_itself = current_treatment  
        #also modifying dataframes to remove baseline acne severities, as they've already been used
        metadata_DFs.append(severities_df[days_of_intro[0][2]:])

    return metadata_DFs

In [526]:
def find_and_plot_severity_states(metadata_DFs):
    """Determines the quantile cutoff determining the quantiles corresponding to categorical acne severity states (low, medium, and high), by
    1) Computing the KDE of the distribution of all normalized acne severity scores over all patients and treatment histories.
    2) Using optimization to find the saddle point of the distribution and consolidating with the modes."""
    
    fig, axes = plt.subplots(1, figsize = (5, 5))
    
    
    #collecting all severities, converting all to positive values, flattening as we go
    all_severities = []
    for df in metadata_DFs:
        all_severities.extend(df["AcneSeverity"] * -1)
    
    #check for normal character by plotting histogram
    severities_histo = np.histogram(all_severities, density = True)
    #axes.hist(all_severities, bins  = 30, density = True)
    axes.set_title("Acne Severities Distribution (relative to baseline)") 
    axes.set_xlabel("Normalized Severity Score")
    axes.set_ylabel("Density")
    
    
    #fitting a kernel density estimate to the data
    
    sns.kdeplot(all_severities, fill=True)
    plt.title("KDE of Acne Severity Distribution")
    
    #extracting the equation of the pdf and finding the local minimum in between the two modes
    kde_pdf = sp.stats.gaussian_kde(all_severities)
    
    #using max and min of pdf to find saddle point in between 2 modes, sampling 1000 points
    severity_grid = np.linspace(np.min(all_severities), np.max(all_severities), 1000)
    
    neg_kde = lambda x: -kde_pdf(x.reshape(1, -1))
    
    #finding the two main modes using optimization, with first 2 mode guesses at the .2 and .8 quantiles
    guesses = np.percentile(all_severities, [20, 80]) 
    
    modes = []
    for guess in guesses:
        better = optimize.minimize(neg_kde, np.array([guess])) 
        modes.append(better.x[0]) 
    
    modes = np.array(modes)
    
    #finding the saddle point in between the two modes, using that as cutoff for the two patient states
    
    initial_guess = np.mean(modes) #average of the modes
    bds = [(min(modes)+1, max(modes)-1)]  # This is a list of two tuples for each mode
    
    saddle_pt = optimize.minimize(kde_pdf, [initial_guess], bounds = bds) 
    state_ranges = [modes[0], saddle_pt.x[0], modes[1]]
    state_names = ["High Severity", "Medium Severity", "Low Severity"]
    
    
    #plotting modes and saddle point over the distribution
    plt.scatter(modes[0], kde_pdf(modes[0]), color = "red", label = "Lower Mode")
    plt.scatter(modes[1], kde_pdf(modes[1]), color = "red", label = "Upper Mode")
    plt.scatter([saddle_pt.x[0]], kde_pdf([saddle_pt.x[0]]), color='green', label="Saddle Point")
    
    #plt.show()
    plt.close()

    return state_names, state_ranges

In [527]:
def assign_states_to_mdfs(metadata_DFs, state_names, state_ranges):
    """Function to construct and attach a second metadata column onto each patient's dataframe. The column contains
    the acne severity categorical state corresponding to a given treatment history."""
    
    for patient_df in metadata_DFs:
        severity_states = np.digitize(-1 * patient_df["AcneSeverity"], state_ranges, right = False)
        severity_states = np.minimum(severity_states, 2)
        patient_df["State"] = [state_names[severity_state] for severity_state in severity_states] 

    return metadata_DFs

In [528]:
def build_histograms(metadata_DFs):
    """This algorithm iterates over all patients' severity dataframes and, for each... 

    1) Counts all of the occcurences of each acne severity state throughout all patients, and assigns those counts 
    as a value to the treatment history key in the state counts dictionary.
    2) Normalizes the counts into distributions before returning both. 
    """
    
    all_state_counts = defaultdict(Counter)
  
    for i, patient_df in enumerate(metadata_DFs):
        histories = patient_df["treatment_history"].values
        states = patient_df["State"].values
 
        for state_index in range(1, len(states)):
            current_state = states[state_index] #state at position state_index in the patient's dataframe
            current_history = histories[state_index] #metadata (treatment history) at position state_index  in the patient's dataframe
            current_history_key = tuple((str(treatment), int(days)) for treatment, days in current_history)
           
            #recording the actual counts of severities, with the context of the prior treatment as the key
            all_state_counts[current_history_key][current_state] += 1

    #normalizing counts dictionaries into distributions
    first_order_probabilities = {}
    
    for previous_treatment, state_counts in all_state_counts.items():
        total_counts  = sum(state_counts.values())
        probabilities_given_previous_state = {state: count/total_counts for state, count in state_counts.items()}
        first_order_probabilities[previous_treatment] = probabilities_given_previous_state

    return (all_state_counts, first_order_probabilities)

In [529]:
def plot_histograms(first_order_probabilities):
    """ This function plots a striped heatmap to inspect the distributions of normalized acne severity states for each treatment history.
    It uses a viridis heatmap implementation from my other repository figuresAndViewers.
    In lieu of using the actual treatment histories themselves as x labels, the x label is the index of the history in the sequence.
    """
    
    x_dim  = 100
    y_dim = 200
    
    fig = plt.figure(figsize=(x_dim, y_dim))
    matplotlib.rc('xtick', labelsize=14)
    matplotlib.rc('ytick', labelsize=14)

      
    #plotting histograms of each context dependent model of acne treatment severity
    bar_width = 0.3
    spacing = 0.05 
    
    #sorting the distributions by the state name in reverse 
    
    real_first_order_probabilities = dict(sorted(first_order_probabilities.items(), reverse = True))
    #print("reals", real_first_order_probabilities)
    
    mainPanelHeight = 15
    mainPanelWidth = 20

    legendPanelHeight = .25
    legendPanelWidth = .5
    
    sidePanelHeight = 3
    sidePanelWidth = .25
    
    
    #setting up the panels and placing the proper positions
    firstMainPanel = plt.axes([.05/x_dim,.375/y_dim, mainPanelWidth/x_dim, mainPanelHeight/
    y_dim])
    firstMainPanel.set_xlabel("Treatment History Index")

    #setting up the legend panel
    legendRight = plt.axes([(1+mainPanelWidth)/x_dim, legendPanelHeight/y_dim, legendPanelWidth/x_dim, mainPanelHeight/y_dim])
    #seting ticks of legend
    legendRight.tick_params(bottom=False, labelbottom=False, left=True, labelleft=True, right=False, labelright=False, top=False, labeltop=False)
    
    legendRight.set_xlim(0,.1)
    legendRight.set_ylim(0,20)
    legendRight.set_yticks([0,20],['0','1'])
    
 
    #looping through to construct a heatmap for all distributions
    entries = len(real_first_order_probabilities)
    bar_width = 1 / entries
    firstMainPanel.set_xlim(0, 1)
    firstMainPanel.set_ylim(0, 1)

    x_pos = 0
    for history, raw_distribution in real_first_order_probabilities.items():
        distribution = defaultdict(float, raw_distribution)
        scaled_x_pos = x_pos * bar_width

        high_fc = viridis(distribution["High Severity"])[:3]
        med_fc  = viridis(distribution["Medium Severity"])[:3]
        low_fc  = viridis(distribution["Low Severity"])[:3]
    

        firstMainPanel.add_patch(rect([scaled_x_pos, 2/3], width=bar_width, height=1/3, facecolor=high_fc, edgecolor='black', linewidth=0.25))
        firstMainPanel.add_patch(rect([scaled_x_pos, 1/3], width=bar_width, height=1/3, facecolor=med_fc, edgecolor='black', linewidth=0.25))
        firstMainPanel.add_patch(rect([scaled_x_pos, 0],   width=bar_width, height=1/3, facecolor=low_fc, edgecolor='black', linewidth=0.25))

        x_actual_spot = scaled_x_pos + bar_width / 2

        x_pos += 1
    num_rects = len(real_first_order_probabilities)
    tick_positions = [i * bar_width + bar_width / 2 for i in range(num_rects)]

    firstMainPanel.set_xticks(tick_positions)
    firstMainPanel.set_xticklabels(['' for _ in tick_positions])
    #firstMainPanel.set_yticks([1/6, 0.5, 5/6], ["Low Severity", "Medium Severity", "High Severity"], fontsize = 15)
    firstMainPanel.set_yticks([1/6, 0.5, 5/6])
    firstMainPanel.set_yticklabels(["Low Severity", "Medium Severity", "High Severity"], fontsize=15)
        
    #plotting viridis heatmap in the sidebar
    #color map tuple pair linspaces, viridis values
    vvLin1Red = np.linspace(68/255, 59/255, 5)
    vvLin2Red = np.linspace(59/255, 33/255, 6)
    vvLin3Red = np.linspace(33/255, 94/255, 6)
    vvLin4Red = np.linspace(94/255, 253/255, 6)
    
    
    vvLin1Green = np.linspace(1/255, 82/255, 5)
    vvLin2Green = np.linspace(82/255, 145/255, 6)
    vvLin3Green = np.linspace(145/255, 201/255, 6)
    vvLin4Green = np.linspace(201/255, 231/255, 6)
    
    vvLin1Blue = np.linspace(84/255, 139/255, 5)
    vvLin2Blue = np.linspace(139/255, 140/255, 6)
    vvLin3Blue = np.linspace(140/255, 98/255, 6)
    vvLin4Blue = np.linspace(98/255, 37/255, 6)
    
    
    plLin4Red = np.linspace(245/255, 237/255, 5)
    plLin3Red = np.linspace(190/255, 245/255, 6)
    plLin2Red = np.linspace(87/255, 190/255, 6)
    plLin1Red = np.linspace(15/255, 87/255, 6)
    
    plLin4Green = np.linspace(135/255,252/255, 5)
    plLin3Green = np.linspace(48/255, 135/255, 6)
    plLin2Green = np.linspace(0/255, 48/255, 6) 
    plLin1Green = np.linspace(0/255, 0/255, 6)
    
    plLin4Blue = np.linspace(48/255, 27/255, 5)
    plLin3Blue = np.linspace(101/255, 48/255,  6)
    plLin2Blue = np.linspace(151/255, 101/255, 6)
    plLin1Blue = np.linspace(118/255, 151/255, 6)
    
    
    #total linspaces for all tuple pairs, viridis values
    vvListOfRedLins = list(vvLin1Red)+list(vvLin2Red)+list(vvLin3Red)+list(vvLin4Red)
    vvListOfGreenLins = list(vvLin1Green)+list(vvLin2Green)+list(vvLin3Green)+list(vvLin4Green)
    vvListOfBlueLins = list(vvLin1Blue)+list(vvLin2Blue)+list(vvLin3Blue)+list(vvLin4Blue)
    
    orderedVVRed = list(dict.fromkeys(vvListOfRedLins))
    orderedVVGreen = list(dict.fromkeys(vvListOfGreenLins))
    orderedVVBlue = list(dict.fromkeys(vvListOfBlueLins))
    
    
    #viridis heatmaps into the legend panel
    for index in range(0,20,1):
    	colorPaletteVV = (orderedVVRed[index], orderedVVGreen[index], orderedVVBlue[index])
    	vvGradeRect = rect([0,index], .1, 8, facecolor=colorPaletteVV,edgecolor = 'black', linewidth = 0)		
    	legendRight.add_patch(vvGradeRect)

    
    #plt.show()
    plt.close()

In [530]:
def build_Dirichlet(prior, all_transition_counts):
    """This function does the following: 
    Accepts a prior in order to construct a Bayesian model of each history's distribution as a Dirichlet distribution with a multinomial 
    likelihood. It does so by the following methods. 
    1)  Iterates through each set of counts for each history and adds them to each parameter in order in the prior, then uses these to
    build the posterior Dirichlet distribution."""

    categories = ['Low Severity', 'Medium Severity', 'High Severity']
    prior_dict = {'Low Severity': prior[0], "Medium Severity": prior[1], "High Severity": prior[2]}
    
    history_and_posteriors = {}

    
    for history, count_dict in all_transition_counts.items():
        
        counts = [count_dict.get(cat, 0) for cat in categories] #pulling counts from prior dict and counts dict
        prior = [prior_dict[cat] for cat in categories] 

        #updating parameters for posterior distribution
        posterior_params = np.array(counts) + np.array(prior)
            
        
        #saving posterior params to the dictionary
        history_and_posteriors[history] = posterior_params
        
    return history_and_posteriors, categories   

In [531]:
def find_dirichlet_marginal_cis(alphas, confidence_level = .95):
    """Function that is wrapped by the below function. Calculates the upper and lower quantiles supplied by confidence interval
    for each marginal beta distribution of a given dirichlet. Returns each confidence interval indexed by the respective alpha."""
    confidence_intervals = {}
    top_density = (1 - confidence_level)/2
    bottom_density = 1 - top_density
    total_alpha = np.sum(alphas)
    

    for index, alpha in enumerate(alphas):
        other_alpha_sum = total_alpha - alpha
        lower_bound = beta.ppf(bottom_density, alpha, other_alpha_sum)
        upper_bound = beta.ppf(top_density, alpha, other_alpha_sum)
        confidence_intervals[index] = (lower_bound, upper_bound)

    return confidence_intervals
        
    

In [532]:
def plot_Dirichlets_credible_interals(histories_and_dirichlets, ordered_categories, confidence_level = .95):
    """This function finds the 95% credible intervals for each component of the Dirichlet distribution for all treatment histories.
    It ensures that a stacked plot is made."""
    
    #checking to see if no categories are supplied; just uses indices of each state to name categories in that case
    if ordered_categories is None:
        ordered_categories = [f"State {i}" for i in range(len(dirichlet_posteriors[0]))]
    
    fig, ax = plt.subplots(len(ordered_categories), 1, figsize=(len(ordered_categories) * 3.5, 7), sharex = True)
    ax[-1].set_xlabel("History Index")

    jitter_spacing = .5
    left_end = 0 
    for history, alphas in histories_and_dirichlets.items():
        
        these_confidence_intervals = find_dirichlet_marginal_cis(alphas)
        for subplot_index, ordered_confidence_interval in these_confidence_intervals.items():
            #ax[subplot_index].set_xticks(np.arange(len(histories_and_dirichlets)))
            #ax[subplot_index].set_xticklabels(list(histories_and_dirichlets.keys()))
            x = left_end + (subplot_index - len(these_confidence_intervals)/2) * jitter_spacing  # adding some x offset for the error bars
            center = (ordered_confidence_interval[1] + ordered_confidence_interval[0])/2
            width = ordered_confidence_interval[0] - ordered_confidence_interval[1]
            
            ax[len(ordered_categories) - subplot_index - 1].errorbar(x, center, yerr = width/2, fmt='o', color='C0', capsize=5)
            
        left_end += 1
            
    for i, name in enumerate(reversed(ordered_categories)):
        ax[i].set_ylabel(name)
            
    plt.tight_layout()
    #plt.show()
    plt.close()

In [533]:
def calculate_KL_Divergence_vectorized(histories_and_posteriors):
    """This vectorized function calculates the Kullback-Leibler divergence between adjacent 
    3 dimensional Dirichlet distributions, each of which is indexed by its alpha parameter.
    It does this for a full array, computing KL Divergence for each term alpha[i] and alpha[i+1]. So, it requires 
    you to supply two arrays of parameters, but really, just the same one read forwards and backwards.
    """
    history_labels = list(histories_and_posteriors.keys())
    x_vals = np.arange(1, len(history_labels))

    alphas = np.array(list(histories_and_posteriors.values()))
    alphas_backward = alphas[:-1]
    alphas_forward = alphas[1:]

    #ensuring alphas are non-0 up front
    sum_forward = np.sum(alphas_forward, axis=1)
    sum_backward = np.sum(alphas_backward, axis=1)

    first_term = gammaln(sum_forward) - gammaln(sum_backward)
    second_term = np.sum(gammaln(alphas_backward) - gammaln(alphas_forward), axis=1)
    third_term = np.sum(
        (alphas_forward - alphas_backward) * 
        (psi(alphas_forward) - psi(sum_forward)[:, None]),
        axis=1
    )

    kl_div = first_term + second_term + third_term
    cumulative_kl = np.cumsum(kl_div)
    cumulative_kl = np.clip(cumulative_kl, 1e-10, None)  # Avoid log(0)

    return x_vals, np.log(cumulative_kl), cumulative_kl

In [534]:
def compute_log_likelihood(points, predictions):
    """This function is wrapped by fit_piecewise_regression. It uses maximum-likelihood estimation
    to estimate the MLE of variance in the residuals for a linear regression model. 
    It then uses the sample size of the points from the model to find the log likelihood of 
    the actual points given the model fit to it."""

    residuals = np.array(points - predictions)
    mle_variance_residuals = (1/len(points)) * np.sum((residuals ** 2))
    log_likelihood = (-len(points)/2) * (np.log10(2*np.pi) + np.log(mle_variance_residuals) + 1) 

    return log_likelihood

In [535]:
def split_for_piecewise_regression(xs, cumulative_kls, percentile_cutoff = 80, split_index = None, gaussian_sigma = 1, slope_threshold = 0.2,
                            min_segment_length = 3):
    """This algorithm uses a brute force method to find the inflection points to fit piecewise regression models to the data.
    But first...
    It dynamically chooses the right number of inflection points to fit each model to via the following...
    1) It smoothes the cumulative KL divergence array with a Gaussian filter (by default is standard normal). 
    2) It computes pairwise slopes between adjacent cumulative KL values (with each x interval = 1/number of steps in treatment history)
    ^check that later
    3) Computes differences between adjacent slopes, returning a 2nd derivative approximation for the entire array
    4) (Currently commented out) Plots the distribution of the magnitudes for inspection.
    5) A cutoff for inflection points is chosen as a percentile of the 2nd derivative magntitdes (default is 90th). 

    Then, it iterates through the found inflection points, dividing them into subarrays of consecutive points. Once it does this, it does 
    one final check to see if each subarray should be further divided. It does this by iterating through each array, comparing the difference in 
    adjacent slopes pairwise, checking their difference against the provided slope threshold parameter. It then makes a list of lists of 
    inflection points where the regression models should start and end.

    It then uses the result of this to further divide the points into subarrays that a regression model can be fit to.
    At last, it checks the length of each subarray against the minimum segment length, and discards ones that are too small.
    
    """

    smoothed_kls = gaussian_filter1d(cumulative_kls, sigma = gaussian_sigma)
    slopes = np.diff(smoothed_kls)
    second_derivatives_magnitudes = np.abs(np.diff(slopes))
    threshold = np.percentile(second_derivatives_magnitudes, percentile_cutoff)
    inflection_points = np.where(second_derivatives_magnitudes > threshold)[0] + 1

    differences = np.diff(inflection_points)
    non_consecutive_indices = np.where(differences != 1)[0] + 1 
    separated_inflection_points = np.array_split(inflection_points, non_consecutive_indices)
    
    #adding the end of the array to the non_consecutive indices for easier splitting later
    separated_inflection_points[-1] = np.append(separated_inflection_points[len(separated_inflection_points)-1], len(smoothed_kls)-1)    
    all_breaks = []

    
    for consecutives_array in separated_inflection_points:
        index_consecutives_array = 0
        last_inflection_pt = 0 
        last_slope = 0

        consecutives_breaks = []
        
        for an_inflection_pt in consecutives_array:
            if an_inflection_pt >= len(slopes):
                continue
            slope_current = slopes[an_inflection_pt]

            slope_difference = np.abs(slope_current - last_slope)
            
            
            if slope_difference > slope_threshold:
                consecutives_breaks.append(an_inflection_pt)
            
            last_slope = slope_current
            last_inflection_pt = an_inflection_pt

        all_breaks.append((consecutives_breaks, index_consecutives_array))

        index_consecutives_array += 1 
   

    fixed_consecutives = []
    for breaks, index in all_breaks:
        
        
        if len(breaks) != 0:
            consecutives_array_to_split = separated_inflection_points[index]
            last_breaking_index = breaks[0] - 1
            for breaking_index in breaks:
                
                broken_consecutives_array = consecutives_array_to_split[last_breaking_index: breaking_index+1]
                fixed_consecutives.append(broken_consecutives_array)
                last_breaking_index = breaking_index
    
    #clipping the appropriate array in the inflection point array of arrays 
    clipped_inflection_points = []
    last_consecutive_break = None

    # Find the last break if it exists
    if fixed_consecutives:
        last_consecutive_break = fixed_consecutives[-1][-1]

    # Search for that break in the separated inflection points
    where_clipping = None
    for i, again_consecutives_array in enumerate(separated_inflection_points):
        if last_consecutive_break in again_consecutives_array:
            idx = np.where(again_consecutives_array == last_consecutive_break)[0][0]
            clipped_inflection_points.append(again_consecutives_array[idx + 1:])
            where_clipping = i
            break  # stop after finding the first match

    # Combine final splits
    if where_clipping is not None:
        full_splits = fixed_consecutives + clipped_inflection_points + separated_inflection_points[where_clipping+1:]
    else:
        full_splits = fixed_consecutives

    return full_splits

In [536]:
def plot_piecewise_regression_segments(curve, splits, ax, color = "blue", linewidth = 2):
    """This function is wrapped by piecewise_regression_and_plot below. It plots the actual line segments piecewise
    over a rendered cumulative KL Divergence plot. """

    if splits is None:
        print("No splits to plot!")
        return
    
    for split in splits:
        split_start, split_end = split[0], split[len(split)-1]
        y_segment = curve[split_start: split_end + 1] 
        x_segment = np.arange(split_start, split_end+1).reshape(-1, 1)

        regression_piece = LinearRegression().fit(x_segment, y_segment.reshape(-1, 1))
        y_pred = regression_piece.predict(x_segment)
        ax.plot(x_segment.flatten(), y_pred.flatten(), color=color, linewidth=linewidth)

In [537]:
def make_KL_Divergence_plot(x_vals, kl_divergences, cumulative_kl, splits):
    """This function plots the Kullback-Leibler Divergence between each consecutive Dirichlet posterior distribution
    as a line plot. This is a wrapper of calculate_KL_Divergence_vectorized.
    It also plots the cumulative KL Divergence over the KL divergence between individual distributions.
    It ends up plotting the linear regression models over the plot as well. """
   
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 5))
    linear_models = plot_piecewise_regression_segments(cumulative_kl, splits, ax, linewidth = 1)
    ax.semilogy(x_vals[0:], kl_divergences[0:], marker='o', label='KL Divergence')
    ax.semilogy(x_vals[0:], cumulative_kl[0:], marker='o', linestyle='--', color='orange', label='Cumulative KL Divergence', alpha = .5)

    ax.set_title('KL Divergence and Cumulative Information Gain (Log Scale)')
    ax.set_xlabel('Treatment History Index')
    ax.set_ylabel('KL Divergence (log scale)')
    ax.legend()
    plt.tight_layout()
    #plt.show()
    plt.close()
    return x_vals, np.log(cumulative_kl)

In [538]:
#retrieving Acne04 dataset from Kaggle source (local download on machine)
raw_data = pd.read_csv("archive (2)/sim_acne.csv")

#actual implementation
this_seperate_DFs, this_intro_days = seperate_patients(raw_data)
this_md_DFs = add_history_metadata(this_seperate_DFs, this_intro_days)
these_states, these_ranges = find_and_plot_severity_states(this_md_DFs)



these_assigned_md_DFs = assign_states_to_mdfs(this_md_DFs, these_states, these_ranges)
built_histograms, raw_probabilities = build_histograms(these_assigned_md_DFs)

plotted_histogram = plot_histograms(raw_probabilities)

uninformative_prior = [1,1,1] #with a1 corresponding to low severity, a2 corresponding to medium, and a3 corresponding to high

these_Dirichlets, these_categories = build_Dirichlet(uninformative_prior, built_histograms)

dirichlet_credible_intervals = plot_Dirichlets_credible_interals(these_Dirichlets, these_categories)

these_xs, this_log_cum_kls, this_kls = calculate_KL_Divergence_vectorized(these_Dirichlets)
splits = split_for_piecewise_regression(these_xs, this_log_cum_kls)
final_plot = make_KL_Divergence_plot(these_xs, this_log_cum_kls, this_kls, splits)




In [539]:
def fit_piecewise_regression_and_plot_unused(points, splits):
    """This function actually splits a given array (here, cumulative KL divergence) into seperate regression models,
    using splits to partition the array and then fit a linear regression model to each one.
    It then calls plot_piecewise_regression_segments to plot the segments over the cumulative KL divergence Curve. Unused in this version."""
    split_points_and_indices = [(points[split[0]:split[1]+1], split) for split in splits]
    linear_model = LinearRegression()
    consecutive_models = [LinearRegression().fit(np.arange(len(one_split_points[0])).reshape(-1, 1),one_split_points[0].reshape(-1, 1))for one_split_points in split_points_and_indices]

    slopes = [which_model.coef_[0] for which_model in consecutive_models]
    

In [540]:
def old_build_histograms(metadata_DFs):
    """Unused in this version. Will condition distributions on previous state and treatment history. Stay tuned... 
    
    
    1) The algorithm iterates over all patient's severity dataframe and, for each... 
    
    Uses the metadata column to build an actual sequence of treatments for each patient along with the state at each. 
    It does this by pulling the treatment history column for each dataset as well as the states, both as lists. Then, it uses a default dict to
    count the occurences of transition from state to state for the current treatment and the one before it. In other words, a dictionary
    of dictionaries is returned, where the keys are the treatments in order of occurence, and the values are dictionaries of counts 
    for each state in the treatment order. 
    
    This amounts to a context-dependent conditional model of acne treatement severity. 
    
    A context-dependent First Order Markov Model is also built in the second loop, using instead a tuple of the previous state and preivous 
    treatment as a key in the default dictionary. 
    
    2) Then, histograms of the context-dependent conditional model of acne treatment severity are plotted. After this, a 3 State visual
    Markov Model is generated for the 1st order Markov Model. 
    
    """
    
    all_state_counts = defaultdict(Counter)
    #all_transition_counts_second_order = defaultdict(Counter)
    
    for i, patient_df in enumerate(metadata_DFs):
        histories = patient_df["treatment_history"].values
        states = patient_df["State"].values

        
        
        for state_index in range(1, len(states)):
            current_state = states[state_index] #state at position state_index in the patient's dataframe
            #previous_history = histories[state_index - 1] #metadata (treatment history) at position state_index - 1 in the patient's dataframe
            current_history = histories[state_index] #metadata (treatment history) at position state_index  in the patient's dataframe
            current_history_key = tuple((str(treatment), int(days)) for treatment, days in current_history)
           
            
            #previous_treatment = previous_history[-1][0] #previous treatment right before position state_index
            #current_treatment = current_history[-1][0] #current treatment at state_index
    
            #recording the actual counts of severities, with the context of the prior treatment as the key
            all_state_counts[current_history_key][current_state] += 1
            
        
        #doing the same for the First order Markov Chain (UNUSED)
        # for state_index in range(2, len(states)):
        #     last_state = states[state_index - 1] 
        #     this_current_state = states[state_index] #state at position state_index in the patient's dataframe
        #     this_previous_history = histories[state_index - 1] #metadata (treatment history) at position state_index - 1 in the patient's dataframe
            
        #     this_current_history = histories[state_index] #metadata (treatment history) at position state_index  in the patient's dataframe
        #     this_previous_treatment = this_previous_history[-1][0] #previous treatment right before position state_index
        #     this_current_treatment = this_current_history[-1][0] #current treatment at state_index
    
        #     this_key = (this_previous_treatment, last_state)
        #     #recording the actual counts of severities, with the context of the prior treatment as the key
        #     all_transition_counts_second_order[this_key][current_state] += 1
            
            
            
    #normalizing counts dictionaries into distributions
    first_order_probabilities = {}
    second_order_probabilities = {}
    
    for previous_treatment, state_counts in all_state_counts.items():
        total_counts  = sum(state_counts.values())
        probabilities_given_previous_state = {state: count/total_counts for state, count in state_counts.items()}
        first_order_probabilities[previous_treatment] = probabilities_given_previous_state
        
        
    # for (previous_treatment, previous_state), state_counts in all_transition_counts_second_order.items(): (UNUSED)
    #     total_counts  = sum(state_counts.values())
    #     probabilities_given_previous_state = {state: count/total_counts for state, count in state_counts.items()}
    #     second_order_probabilities[(previous_treatment, previous_state)] = probabilities_given_previous_state

    print(all_state_counts)

    return (all_state_counts, first_order_probabilities) #, (all_transition_counts_second_order, second_order_probabilities)